In [1]:
from codeinterpreter.sandbox.local_jupyter.manager import LocalJupyterManager

In [2]:
manager = LocalJupyterManager()
manager.init()
sandbox =  manager.start()

In [3]:
from pydantic import BaseModel
from langchain.tools import StructuredTool

def code_run(code:str):
    output =  sandbox.run(code=code)
    return output.content

class CodeInput(BaseModel):
    code: str

tools = [
    StructuredTool(
        name="python",
        description="Input a string of code to a ipython interpreter. "
        "Write the entire code in a single string. This string can "
        "be really long, so you can use the `;` character to split lines. "
        "Variables are preserved between runs. ",
        func=code_run,
        args_schema=CodeInput,
    )
]



In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from codeinterpreter.agent.openai_functions_agent.base import CustomOpenAIFunctionsAgent


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

agent = CustomOpenAIFunctionsAgent.from_llm_and_tools(llm=llm,tools=tools)

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True
            ), verbose=True)

In [5]:
result = agent_executor({"input": "print two random number"})
result



> Entering new AgentExecutor chain...

Invoking: `python` with `import random

# Generate two random numbers
random_number1 = random.randint(1, 100)
random_number2 = random.randint(1, 100)

# Print the random numbers
print(random_number1, random_number2)`


93 27The two random numbers are 93 and 27.

> Finished chain.


{'input': 'print two random number',
 'chat_history': [HumanMessage(content='print two random number', additional_kwargs={}, example=False),
  AIMessage(content='The two random numbers are 93 and 27.', additional_kwargs={}, example=False)],
 'output': 'The two random numbers are 93 and 27.'}

In [6]:
result = agent_executor({"input": "get this computer name"})
result



> Entering new AgentExecutor chain...

Invoking: `python` with `import socket

# Get the computer name
computer_name = socket.gethostname()

computer_name`


'arcosx-win'The computer name of this machine is "arcosx-win".

> Finished chain.


{'input': 'get this computer name',
 'chat_history': [HumanMessage(content='print two random number', additional_kwargs={}, example=False),
  AIMessage(content='The two random numbers are 93 and 27.', additional_kwargs={}, example=False),
  HumanMessage(content='get this computer name', additional_kwargs={}, example=False),
  AIMessage(content='The computer name of this machine is "arcosx-win".', additional_kwargs={}, example=False)],
 'output': 'The computer name of this machine is "arcosx-win".'}

In [7]:
result =  agent_executor({"input": "read this page https://simple.wikipedia.org/wiki/Evolution and get title for me"})
result



> Entering new AgentExecutor chain...

Invoking: `python` with `{'code': "import requests\nfrom bs4 import BeautifulSoup\n\nurl = 'https://simple.wikipedia.org/wiki/Evolution'\nresponse = requests.get(url)\nsoup = BeautifulSoup(response.content, 'html.parser')\ntitle = soup.find('h1', {'id': 'firstHeading'}).text\n\ntitle"}`


'Evolution'The title of the page is "Evolution".

> Finished chain.


{'input': 'read this page https://simple.wikipedia.org/wiki/Evolution and get title for me',
 'chat_history': [HumanMessage(content='print two random number', additional_kwargs={}, example=False),
  AIMessage(content='The two random numbers are 93 and 27.', additional_kwargs={}, example=False),
  HumanMessage(content='get this computer name', additional_kwargs={}, example=False),
  AIMessage(content='The computer name of this machine is "arcosx-win".', additional_kwargs={}, example=False),
  HumanMessage(content='read this page https://simple.wikipedia.org/wiki/Evolution and get title for me', additional_kwargs={}, example=False),
  AIMessage(content='The title of the page is "Evolution".', additional_kwargs={}, example=False)],
 'output': 'The title of the page is "Evolution".'}